## WMT17 Data Preprocessing

Update about the data processing:
1. Solved the machine translation sentence and human score dismatching issue.
    1. Method: Previously accidentally doing the list slicing with wrong slicing number.
2. Solved the machine translation system name dismatch issue.
    1. Method: For machine translation system name like 'jhu-nmt.5153+xmunmt.5165', 'jhu-nmt.5153' and 'xmunmt.5165' have the exact the machine translation output. So, we only take the first machine translation system name into account, and extract the corresponding machine transaltion sentences. The human score should be the same for both mt system.
3. The data should be fine now.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
'''
                                       ####IMPORTANT####
   Data process goal: If you open the raw data zip file, you will find a file name as "DA-seglevel". 
   The file's information is showing in the section of "wmt17_new_df" below, which has been printed out.
   Our goal is to:
       1. based on the LP name, SID, and the SYSTEM name to find the corresponding source sentence, reference sentence, and machine translation
       sentences, and integrate the human score correspondantly.
       2. The output data structure would be: {sentence id: [src sent, ref sent, [mt , human score]]}.
       3. Due to the there are many system name, SID missing, and other problems of the dataset, there is a big gap exists between the number
       of human score and the source/reference sentences.
       4. The explanation of LP, DATA, SYSTEM, SID, HUMAN score are specified below.
'''

'\n                                       ####IMPORTANT####\n   Data process goal: If you open the raw data zip file, you will find a file name as "DA-seglevel". \n   The file\'s information is showing in the section of "wmt17_new_df" below, which has been printed out.\n   Our goal is to:\n       1. based on the LP name, SID, and the SYSTEM name to find the corresponding source sentence, reference sentence, and machine translation\n       sentences, and integrate the human score correspondantly.\n       2. The output data structure would be: {sentence id: [src sent, ref sent, [mt , human score]]}.\n       3. Due to the there are many system name, SID missing, and other problems of the dataset, there is a big gap exists between the number\n       of human score and the source/reference sentences.\n       4. The explanation of LP, DATA, SYSTEM, SID, HUMAN score are specified below.\n'

In [3]:
#data frame of the WMT17
wmt17_df = pd.read_csv("D:\IC Research\wmt17-metrics-task\DA-seglevel.csv")

In [4]:
wmt17_df_keys = wmt17_df.keys()[0].split(" ")
print(wmt17_df_keys)

['LP', 'DATA', 'SYSTEM', 'SID', 'HUMAN']


In [5]:
wmt17_df_values = np.array([i[0].split(" ") for i in wmt17_df.values]).transpose().tolist()

In [6]:
wmt17_new_df = {}
lp = ["".join(i.split('-')) for i in wmt17_df_values[0]]

wmt17_df_values[0] = lp

wmt17_new_df[wmt17_df_keys[0]] = wmt17_df_values[0]
wmt17_new_df[wmt17_df_keys[1]] = wmt17_df_values[1]
wmt17_new_df[wmt17_df_keys[2]] = wmt17_df_values[2]
wmt17_new_df[wmt17_df_keys[3]] = wmt17_df_values[3]
wmt17_new_df[wmt17_df_keys[4]] = wmt17_df_values[4]

#newly processed dataframe
wmt17_new_df = pd.DataFrame(wmt17_new_df)

In [7]:
print(wmt17_new_df)

'''
1. LP: the language pair.
2. DATA: the data source.
3. SYSTEM: the machine translation system name which translates source sentence or reference sentence to machine translations.
4. SID: Sentence ID in each file. More precisely, it represents the sentence is at which line of the data. 
   e.g. there are 1000 sentences in a file, and a sentence is appears at line 23 of this file, then the sentence id of this sentence is 23.
5. HUMAN: The human score.
'''

        LP          DATA            SYSTEM   SID               HUMAN
0     enzh  newstest2017      UU-HNMT.5134    11  -0.522268402810163
1     enzh  newstest2017      jhu-nmt.5153    28  -0.150440953127025
2     enzh  newstest2017  CASICT-cons.5157    39  -0.330677981704853
3     enzh  newstest2017        online-A.0    49   0.490614482852024
4     enzh  newstest2017  CASICT-cons.5157    68   -0.43186463367268
...    ...           ...               ...   ...                 ...
5035  enru  newstest2017        online-F.0  2763  -0.671133057028143
5036  enru  newstest2017     jhu-pbmt.4986  2790   -0.55131040507727
5037  enru  newstest2017    uedin-nmt.4756  2803   -1.16214657123657
5038  enru  newstest2017        online-G.0  2913  -0.364504266728924
5039  enru  newstest2017    uedin-nmt.4756  2986   0.591125675642942

[5040 rows x 5 columns]


'\n1. LP: the language pair.\n2. DATA: the data source.\n3. SYSTEM: the machine translation system name which translates source sentence or reference sentence to machine translations.\n4. SID: Sentence ID in each file. More precisely, it represents the sentence is at which line of the data. \n   e.g. there are 1000 sentences in a file, and a sentence is appears at line 23 of this file, then the sentence id of this sentence is 23.\n5. HUMAN: The human score.\n'

In [8]:
#retrive language pair names
unique_lp = wmt17_new_df.LP.unique()

In [9]:
unique_lp

array(['enzh', 'csen', 'deen', 'fien', 'lven', 'ruen', 'tren', 'zhen',
       'enru'], dtype=object)

In [10]:
#retrieve sentence unique sentence ids
unique_id = wmt17_new_df.SID.unique()

In [11]:
unique_id

array(['11', '28', '39', ..., '2633', '2719', '2803'], dtype=object)

In [12]:
print('119' in unique_id)

True


In [13]:
#create DataFrameDict to seperate sub-dataframes for the specific language pairs
DataFrameDict_lp = {elem : pd.DataFrame for elem in unique_lp}

In [14]:
DataFrameDict_lp

{'enzh': pandas.core.frame.DataFrame,
 'csen': pandas.core.frame.DataFrame,
 'deen': pandas.core.frame.DataFrame,
 'fien': pandas.core.frame.DataFrame,
 'lven': pandas.core.frame.DataFrame,
 'ruen': pandas.core.frame.DataFrame,
 'tren': pandas.core.frame.DataFrame,
 'zhen': pandas.core.frame.DataFrame,
 'enru': pandas.core.frame.DataFrame}

In [15]:
#create the sub-dataframe by machine translation sentence lp
for key in DataFrameDict_lp.keys():
    DataFrameDict_lp[key] = wmt17_new_df[:][wmt17_new_df.LP == key]

In [16]:
#sample output for the information of en-zh
DataFrameDict_lp['enzh']

LP          DATA                          SYSTEM   SID  \
0    enzh  newstest2017                    UU-HNMT.5134    11   
1    enzh  newstest2017                    jhu-nmt.5153    28   
2    enzh  newstest2017                CASICT-cons.5157    39   
3    enzh  newstest2017                      online-A.0    49   
4    enzh  newstest2017                CASICT-cons.5157    68   
..    ...           ...                             ...   ...   
555  enzh  newstest2017                      online-B.0  1824   
556  enzh  newstest2017                      online-F.0  1903   
557  enzh  newstest2017                    UU-HNMT.5134  1926   
558  enzh  newstest2017                  uedin-nmt.5111  1952   
559  enzh  newstest2017  Oregon-State-University-S.5174  1980   

                  HUMAN  
0    -0.522268402810163  
1    -0.150440953127025  
2    -0.330677981704853  
3     0.490614482852024  
4     -0.43186463367268  
..                  ...  
555   0.297019960751344  
556  -0.559206505132111  
557  -0.813910057777521  
558   0.348248843834138  
559  -0.753565709943098  

[560 rows x 5 columns]

In [17]:
#create DataFrameDict to seperate sub-dataframes for the specific sentence id
DataFrameDict_id = {elem : pd.DataFrame for elem in unique_id}

In [18]:
#create the sub-dataframe by machine translation sentence id
for key in DataFrameDict_id.keys():
    DataFrameDict_id[key] = wmt17_new_df[:][wmt17_new_df.SID == key]

In [19]:
#sample output given the sentence id as '15'
DataFrameDict_id['15']

LP          DATA                   SYSTEM SID               HUMAN
3710  tren  newstest2017  afrl-mitll-m2w-nr1.4901  15  -0.969731377007766

In [20]:
#sentence id dict for extracting mt sentence and human score
id_dict = {}

for id in DataFrameDict_id:
    id_dict[id] = DataFrameDict_id[id].to_dict() 

In [21]:
#output example of id_dict
id_dict['11']

{'LP': {0: 'enzh', 3150: 'ruen', 4480: 'enru', 4760: 'enru'},
 'DATA': {0: 'newstest2017',
  3150: 'newstest2017',
  4480: 'newstest2017',
  4760: 'newstest2017'},
 'SYSTEM': {0: 'UU-HNMT.5134',
  3150: 'afrl-mitll-opennmt.4896+afrl-mitll-syscomb.4905',
  4480: 'online-H.0',
  4760: 'online-G.0'},
 'SID': {0: '11', 3150: '11', 4480: '11', 4760: '11'},
 'HUMAN': {0: '-0.522268402810163',
  3150: '0.347389541833074',
  4480: '0.538309605782715',
  4760: '1.02517135622773'}}

In [22]:
DataFrameDict_lp['enzh']

LP          DATA                          SYSTEM   SID  \
0    enzh  newstest2017                    UU-HNMT.5134    11   
1    enzh  newstest2017                    jhu-nmt.5153    28   
2    enzh  newstest2017                CASICT-cons.5157    39   
3    enzh  newstest2017                      online-A.0    49   
4    enzh  newstest2017                CASICT-cons.5157    68   
..    ...           ...                             ...   ...   
555  enzh  newstest2017                      online-B.0  1824   
556  enzh  newstest2017                      online-F.0  1903   
557  enzh  newstest2017                    UU-HNMT.5134  1926   
558  enzh  newstest2017                  uedin-nmt.5111  1952   
559  enzh  newstest2017  Oregon-State-University-S.5174  1980   

                  HUMAN  
0    -0.522268402810163  
1    -0.150440953127025  
2    -0.330677981704853  
3     0.490614482852024  
4     -0.43186463367268  
..                  ...  
555   0.297019960751344  
556  -0.559206505132111  
557  -0.813910057777521  
558   0.348248843834138  
559  -0.753565709943098  

[560 rows x 5 columns]

In [23]:
def read_src_tgt(directory, lp):
    #return a dictionary by language pair: {sentence index: [sentence]}
    all_files = os.listdir(directory)
    file_dict = {}
    for i in all_files:
        file_dict[i.split('-')[1]] = directory + "\\" +i
    file_dir = file_dict[lp]
    file = open(file_dir, encoding="utf8")
    
    sent_list = [i.strip() for i in file.readlines()]
    sent_dict = {}
    index = 1
    while index < len(sent_list)+1:
        sent_dict[index] = [lp, sent_list[index-1]]
        index += 1
    return sent_dict

#example output
read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources', 'enzh')


{1: ['enzh', '28-Year-Old Chef Found Dead at San Francisco Mall'],
 2: ['enzh',
  'A 28-year-old chef who had recently moved to San Francisco was found dead in the stairwell of a local mall this week.'],
 3: ['enzh',
  'But the victim\'s brother says he can\'t think of anyone who would want to hurt him, saying, "Things were finally going well for him."'],
 4: ['enzh',
  "The body found at the Westfield Mall Wednesday morning was identified as 28-year-old San Francisco resident Frank Galicia, the San Francisco Medical Examiner's Office said."],
 5: ['enzh',
  'The San Francisco Police Department said the death was ruled a homicide and an investigation is ongoing.'],
 6: ['enzh',
  "The victim's brother, Louis Galicia, told ABC station KGO in San Francisco that Frank, previously a line cook in Boston, had landed his dream job as line chef at San Francisco's Sons & Daughters restaurant six months ago."],
 7: ['enzh',
  'A spokesperson for Sons & Daughters said they were "shocked and devas

In [24]:
#all language pair source sentence dictionary: ['enzh', 'csen', 'deen', 'fien', 'lven', 'ruen', 'tren', 'zhen','enru'

# en-zh
src_enzh_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'enzh')

# cs-en
src_csen_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'csen')

#de-en
src_deen_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'deen')

#fi-en
src_fien_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'fien')

#lv-en
src_lven_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'lven')

#ru-en
src_ruen_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'ruen')

#tr-en
src_tren_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'tren')

#zh-en
src_zhen_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'zhen')

#en-ru
src_enru_sent_dict = read_src_tgt('D:\IC Research\wmt17-metrics-task\input-output-ref-data\sources' ,'enru')

In [25]:
#sample output of src_en-zh_sent, 1000 sentence in total
src_enzh_sent_dict

{1: ['enzh', '28-Year-Old Chef Found Dead at San Francisco Mall'],
 2: ['enzh',
  'A 28-year-old chef who had recently moved to San Francisco was found dead in the stairwell of a local mall this week.'],
 3: ['enzh',
  'But the victim\'s brother says he can\'t think of anyone who would want to hurt him, saying, "Things were finally going well for him."'],
 4: ['enzh',
  "The body found at the Westfield Mall Wednesday morning was identified as 28-year-old San Francisco resident Frank Galicia, the San Francisco Medical Examiner's Office said."],
 5: ['enzh',
  'The San Francisco Police Department said the death was ruled a homicide and an investigation is ongoing.'],
 6: ['enzh',
  "The victim's brother, Louis Galicia, told ABC station KGO in San Francisco that Frank, previously a line cook in Boston, had landed his dream job as line chef at San Francisco's Sons & Daughters restaurant six months ago."],
 7: ['enzh',
  'A spokesperson for Sons & Daughters said they were "shocked and devas

In [26]:
#all language pair tgt sentence dictionary: ['enzh', 'csen', 'deen', 'fien', 'lven', 'ruen', 'tren', 'zhen','enru'

# en-zh
ref_enzh_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'enzh')

# cs-en
ref_csen_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'csen')

#de-en
ref_deen_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'deen')

#fi-en
ref_fien_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'fien')

#lv-en
ref_lven_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'lven')

#ru-en
ref_ruen_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'ruen')

#tr-en
ref_tren_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'tren')

#zh-en
ref_zhen_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'zhen')

#en-ru
ref_enru_sent_dict = read_src_tgt('D:/IC Research/wmt17-metrics-task/input-output-ref-data/references' ,'enru')

In [27]:
#sample output of ref_enzh_sent_dict. 1000 sentences in total
ref_enzh_sent_dict

{1: ['enzh', '28岁厨师被发现死于旧金山一家商场'],
 2: ['enzh', '近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。'],
 3: ['enzh', '但受害人的哥哥表示想不出有谁会想要加害于他，并称“一切终于好起来了。”'],
 4: ['enzh', '旧金山验尸官办公室表示，周三早上于西田购物中心发现的尸体确认为28岁旧金山居民 Frank Galicia。'],
 5: ['enzh', '旧金山警察局称该起死亡案件被裁定为他杀，并正在进行调查。'],
 6: ['enzh',
  '受害人的哥哥 Louis Galicia 对美国广播公司位于旧金山的电台 KGO 表示，之前在波士顿做流水线厨师的Frank 于六个月前在旧金山的 Sons & Daughters 餐馆找到一份流水线厨师的理想工作。'],
 7: ['enzh', 'Sons & Daughters 餐馆的一位发言人表示，他们对于 Frank 的死感到“非常震惊”。'],
 8: ['enzh', '该发言人称，“我们是相处得像亲密无间的一家人的一个小团队，我们将深深怀念他。”'],
 9: ['enzh', '在这个悲痛的时刻，我们向 Frank 的家人及朋友表达我们深切的同情与哀悼。'],
 10: ['enzh', 'Louis Galicia 称 Frank 起初住在招待所里，但是最近“一切终于好起来了。”'],
 11: ['enzh', 'Louis Galicia 告诉 KGO：“Frank找到一间公寓，同时在跟一个女孩交往。”'],
 12: ['enzh', 'Louis Galicia 表示，他想不出有谁会想要加害他的弟弟。'],
 13: ['enzh', 'Frank 拥有善良的心地及宽阔的胸怀。'],
 14: ['enzh', 'Louis Galicia 称：“Frank 经常为家人烹饪晚餐。”'],
 15: ['enzh', '他从不愿意与家人争吵。'],
 16: ['enzh', '他是一个随从大家意见的人。'],
 17: ['enzh', 'Louis Galicia 说：“世上的所有事物都有可能出错，但Frank 称得上是一个每天都发光的外粗内秀之人。”'],
 18: ['enzh', '有线索人士

In [28]:
def cat_src_ref(src, ref):
    #this method returns the concatenated version of the source and reference sentence with its index: {sentence index: [src sent, ref sent]}
    out = {}
    for i in src:
        out[str(i)] = src[i] + [ref[i][1]]
    return out

In [29]:
#concatenate all language source and reference sentence

#en-zh
src_ref_enzh = cat_src_ref(src_enzh_sent_dict, ref_enzh_sent_dict)

#cs-en
src_ref_csen = cat_src_ref(src_csen_sent_dict, ref_csen_sent_dict)

#de-en
src_ref_deen = cat_src_ref(src_deen_sent_dict, ref_deen_sent_dict)

#fi-en
src_ref_fien = cat_src_ref(src_fien_sent_dict, ref_fien_sent_dict)

#lv-en
src_ref_lven = cat_src_ref(src_lven_sent_dict, ref_lven_sent_dict)

#ru-en
src_ref_ruen = cat_src_ref(src_ruen_sent_dict, ref_ruen_sent_dict)

#tr-en
src_ref_tren = cat_src_ref(src_tren_sent_dict, ref_tren_sent_dict)

#zh-en
src_ref_zhen = cat_src_ref(src_zhen_sent_dict, ref_zhen_sent_dict)

#en-ru
src_ref_enru = cat_src_ref(src_enru_sent_dict, ref_enru_sent_dict)

In [30]:
#sample output of src sentence + ref sentence; 1000 sentence in total.
src_ref_enzh

{'1': ['enzh',
  '28-Year-Old Chef Found Dead at San Francisco Mall',
  '28岁厨师被发现死于旧金山一家商场'],
 '2': ['enzh',
  'A 28-year-old chef who had recently moved to San Francisco was found dead in the stairwell of a local mall this week.',
  '近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。'],
 '3': ['enzh',
  'But the victim\'s brother says he can\'t think of anyone who would want to hurt him, saying, "Things were finally going well for him."',
  '但受害人的哥哥表示想不出有谁会想要加害于他，并称“一切终于好起来了。”'],
 '4': ['enzh',
  "The body found at the Westfield Mall Wednesday morning was identified as 28-year-old San Francisco resident Frank Galicia, the San Francisco Medical Examiner's Office said.",
  '旧金山验尸官办公室表示，周三早上于西田购物中心发现的尸体确认为28岁旧金山居民 Frank Galicia。'],
 '5': ['enzh',
  'The San Francisco Police Department said the death was ruled a homicide and an investigation is ongoing.',
  '旧金山警察局称该起死亡案件被裁定为他杀，并正在进行调查。'],
 '6': ['enzh',
  "The victim's brother, Louis Galicia, told ABC station KGO in San Francisco that Frank, previously 

In [31]:
'''
id_dict['11']= 

{'LP': {0: 'enzh', 3150: 'ruen', 4480: 'enru', 4760: 'enru'},
----------------
 'DATA': {0: 'newstest2017',
  3150: 'newstest2017',
  4480: 'newstest2017',
  4760: 'newstest2017'},
------------------  
 'SYSTEM': {0: 'UU-HNMT.5134',
  3150: 'afrl-mitll-opennmt.4896+afrl-mitll-syscomb.4905',
  4480: 'online-H.0',
  4760: 'online-G.0'},
------------------  
 'SID': {0: '11', 3150: '11', 4480: '11', 4760: '11'},
--------------------
 'HUMAN': {0: '-0.522268402810163',
  3150: '0.347389541833074',
  4480: '0.538309605782715',
  4760: '1.02517135622773'}}
'''

def mt_score_extract(directory, id_dict, id, lp, lp_slash):
    #return a nested list: [[mt1, score1], [mt2, score2], ...]
    all_files = os.listdir(directory+ '\\' +lp_slash)
    file_dict = {}
    
    try:

        for i in all_files:
            #print(i)
            system_name = i.split('.')[1:-1]
            if len(system_name)==1:
                system_name = system_name
            else:
                system_name = ".".join(system_name)
            file_dict[system_name] = directory + "\\" + lp_slash + "\\" +i

        #enter the id_dict
        df_dict = id_dict[id]

        #extract the keys in the sub dict
        right_key = [key for key in df_dict['LP'] if df_dict['LP'][key] == lp]
        #print(right_key)

        #extract the right systme name from the right key
        right_sys = []
        for i in right_key:
            if '+' not in df_dict['SYSTEM'][i]:
                right_sys.append(df_dict['SYSTEM'][i])
            else:
                right_sys.append(df_dict['SYSTEM'][i].split("+")[0])
        #right_sys = [df_dict['SYSTEM'][i] for i in right_key]
        #print(right_sys)


        #extract the right human score from the right key
        right_score = [df_dict['HUMAN'][i] for i in right_key]
        #print(right_score)

        #extract the right file path
        right_path = [file_dict[i] for i in right_sys]
        #print(right_path)

        #extract the right sentence based on the right file path
        right_sent = []
        for path in right_path:
            file = open(path, encoding="utf8")
            sent = [i.strip() for i in file.readlines()][int(id)-1]
            #print(sent)
            right_sent.append(sent)

        #combine right sentence and right score together
        assert len(right_sent) == len(right_score)

        result = [[right_sent[index], right_score[index]] for index in range(0, len(right_sent))]
    
    except Exception as e:
        result = []
    
    return result

In [32]:
mt_score_extract('D:/IC Research/wmt17-metrics-task/input-output-ref-data/system-outputs', id_dict, str(103), "zhen", "zh-en")

[]

In [33]:
def final_cat(src_ref_dict ,directory, id_dict):
    #diectory is the root directory: D:/IC Research/wmt17-metrics-task/input-output-ref-data
    #data structure {sentence id: [src sent, ref sent, [mt , human score]]}
    mt_dir = directory + "\\" + "system-outputs"
    result = {}
    #print(src_ref_dict)
    for id in src_ref_dict:
        #print(id)
        dict_value = src_ref_dict[id]
        #print(dict_value)
        lp = dict_value[0]
        #print(lp)
        lp_slash = lp[0:2] + "-" + lp[2:]
        #print(lp_slash)
        
        mt_score = mt_score_extract(mt_dir, id_dict, id, lp, lp_slash)
        
        dict_value.append(mt_score)
        #print(dict_value)
        #add value to result dict
        result[id] = dict_value
    return result

In [34]:
#sample output of the final data
final_cat(src_ref_zhen, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

{'1': ['zhen',
  '28岁厨师被发现死于旧金山一家商场',
  '28-Year-Old Chef Found Dead at San Francisco Mall',
  []],
 '2': ['zhen',
  '近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。',
  'A 28-year-old chef who had recently moved to San Francisco was found dead in the stairwell of a local mall this week.',
  [['A 28 chef, who has just moved to San Francisco, was found dead on the stairs of a local mall this week.',
    '0.578774568361302']]],
 '3': ['zhen',
  '但受害人的哥哥表示想不出有谁会想要加害于他，并称“一切终于好起来了。”',
  'But the victim\'s brother says he can\'t think of anyone who would want to hurt him, saying, "Things were finally going well for him."',
  []],
 '4': ['zhen',
  '旧金山验尸官办公室表示，周三早上于西田购物中心发现的尸体确认为28岁旧金山居民 Frank Galicia。',
  "The body found at the Westfield Mall Wednesday morning was identified as 28-year-old San Francisco resident Frank Galicia, the San Francisco Medical Examiner's Office said.",
  []],
 '5': ['zhen',
  '旧金山警察局称该起死亡案件被裁定为他杀，并正在进行调查。',
  'The San Francisco Police Department said the death was ruled a homic

In [35]:
#all language pair sentence final ouput: ['enzh', 'csen', 'deen', 'fien', 'lven', 'ruen', 'tren', 'zhen','enru'

#enzh
final_enzh = final_cat(src_ref_enzh,  'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#csen
final_csen = final_cat(src_ref_csen, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#deen
final_deen = final_cat(src_ref_deen, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#fien
final_fien = final_cat(src_ref_fien, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#lven
final_lven = final_cat(src_ref_lven, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#ruen
final_ruen = final_cat(src_ref_ruen, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#tren
final_tren = final_cat(src_ref_tren, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#zhen
final_zhen = final_cat(src_ref_zhen, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

#enru
final_enru = final_cat(src_ref_enru, 'D:/IC Research/wmt17-metrics-task/input-output-ref-data', id_dict)

In [36]:
'''
local file path need to be specified.
'''
def save_obj(obj, name ):
    #save dictionary files using this function.
    with open('D:/IC Research/wmt17-processed-data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    #load dictionary files using this function.
    
    with open('D:/IC Research/wmt17-processed-data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [37]:
save_obj(final_enzh, "final_enzh")
save_obj(final_deen, "final_deen")
save_obj(final_csen, "final_csen")
save_obj(final_fien, "final_fien")
save_obj(final_ruen, "final_ruen")
save_obj(final_lven, "final_lven")
save_obj(final_zhen, "final_zhen")
save_obj(final_enru, "final_enru")